In [1]:
%load_ext nb_mypy

Version 1.0.5


In [2]:
from loguru import logger
from s3path import S3Path
from datetime import datetime

from sm_pipelines_oo.steps.framework_processing_step import StepFactory
from sm_pipelines_oo.shared_config_schema import SharedConfig, Environment
from sm_pipelines_oo.aws_connector.concrete_connectors import create_aws_connector
from sm_pipelines_oo.config_loader.implementations.file_loaders import YamlConfigLoader

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/thomas-22/.config/sagemaker/config.yaml


In [3]:
ENV: Environment = 'local'
_config_loader = YamlConfigLoader(
    env=ENV,
    config_root_folder='../../config'  # relative to package root
)
_aws_connector = create_aws_connector(
    shared_config=SharedConfig(**_config_loader.shared_config_as_dict),
    environment=ENV,
)
factory = StepFactory(
    step_config_dict=_config_loader.step_configs_as_dicts[0],  # There is only 1 element
    role_arn=_aws_connector.role_arn,
    sm_session=_aws_connector.sm_session,
    pipeline_session=_aws_connector.pipeline_session,
)

In [4]:
# Create input dataframe (with current date for validation) and save it to S3
from smp_oo_examples.validation import setup_input

input_path_s3 = f"{_config_loader.step_configs_as_dicts[0]['processor_run_config']['inputs']['input_3']}/input.parquet"
validation_data = setup_input(input_path_s3=input_path_s3)

In [5]:
factory.run_processor()

INFO:sagemaker:Creating processing-job with name sklearn-2024-03-22-13-17-21-759
INFO:sagemaker.local.image:'Docker Compose' found using Docker CLI.
INFO:sagemaker.local.local_session:Starting processing job
INFO:sagemaker.local.image:docker compose file: 
networks:
  sagemaker-local:
    name: sagemaker-local
services:
  algo-1-l6p3y:
    container_name: 9drqmxp9jg-algo-1-l6p3y
    entrypoint:
    - /bin/bash
    - /opt/ml/processing/input/entrypoint/runproc.sh
    environment:
    - '[Masked]'
    - '[Masked]'
    - '[Masked]'
    - '[Masked]'
    image: 683313688378.dkr.ecr.us-east-1.amazonaws.com/sagemaker-scikit-learn:0.23-1-cpu-py3
    networks:
      sagemaker-local:
        aliases:
        - algo-1-l6p3y
    stdin_open: true
    tty: true
    volumes:
    - /tmp/tmp9f7i5dst/algo-1-l6p3y/config:/opt/ml/config
    - /tmp/tmp9f7i5dst/algo-1-l6p3y/output:/opt/ml/output
    - /tmp/tmple_vszim:/opt/ml/processing/input_3
    - /tmp/tmps2mn9y2w:/opt/ml/processing/input/code/
    - /tm

time="2024-03-22T09:17:25-04:00" level=warning msg="a network with name sagemaker-local exists but was not created for project \"tmp9f7i5dst\".\nSet `external: true` to use an existing network"
 Container 9drqmxp9jg-algo-1-l6p3y  Creating
 Container 9drqmxp9jg-algo-1-l6p3y  Created
Attaching to 9drqmxp9jg-algo-1-l6p3y
9drqmxp9jg-algo-1-l6p3y  | Found existing installation: typing 3.7.4.3
9drqmxp9jg-algo-1-l6p3y  | Uninstalling typing-3.7.4.3:
9drqmxp9jg-algo-1-l6p3y  |   Successfully uninstalled typing-3.7.4.3
9drqmxp9jg-algo-1-l6p3y  | WARNING: Running pip as the 'root' user can result in broken permissions and conflicting behaviour with the system package manager. It is recommended to use a virtual environment instead: https://pip.pypa.io/warnings/venv
9drqmxp9jg-algo-1-l6p3y  | WARNING: Running pip as the 'root' user can result in broken permissions and conflicting behaviour with the system package manager. It is recommended to use a virtual environment instead: https://pip.pypa.io/

INFO:sagemaker.local.image:===== Job Complete =====


In [6]:
from smp_oo_examples.validation import check_output

_output_s3_dir = _config_loader.step_configs_as_dicts[0]['processor_run_config']['outputs']['output_1']
_output_filename = _config_loader.step_configs_as_dicts[0]['processor_init_config']['env']['OUTPUT_FILENAME']
check_output(
    df_in= validation_data['input'],
    output_path_s3=f'{_output_s3_dir}/{_output_filename}',
)

   a  b                             date
0  2  4 2024-03-22 13:17:36.056474+00:00
1  2  4 2024-03-22 13:17:36.056474+00:00
actual_timedelta: 0 days 00:00:04.669558


In [7]:
print(datetime.now())

2024-03-22 09:17:42.191800
